# PhysiCell 2021 Virtual Workshop and Hackathon
## Session 2: PhysiCell First Dive
* Refresher: Sample and Template Projects 
* Refresher: Project Structure
* Cells, Phenotype, and Cell Defintions
* Learn about general modeling workflow
  * Basic 	(Sessions 1, 2)
  * Intermediate 	(Session 5)
  * Full 	(Sessions 6-12)
* Populate, build, and run a basic model (Basic Workflow)
* Load and visualize data in Python

### Code Section 1

In [ ]:
from pyMCDS import pyMCDS
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Type pyMCDS. and hit the tab key to see methods 

!ls ./output/*.xml 

### Code Section 2

In [ ]:
mcds = pyMCDS('output00000000.xml', 'output')

In [ ]:
print(mcds.get_time())  # what simulation time is saved here? 
print(mcds.get_cell_variables()) # what data are saved in the cells? 
print(mcds.get_substrate_names())  # what diffusing substrates? 

In [ ]:
mcds.data.keys()

### Code Section 3

In [ ]:
np.mean( mcds.data['discrete_cells']['oncoprotein'] )

In [ ]:
import matplotlib 

matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 

plt.hist( mcds.data['discrete_cells']['oncoprotein'] )


### Code Section 4

In [ ]:
cx = mcds.data['discrete_cells']['position_x']
cy = mcds.data['discrete_cells']['position_y']
op = mcds.data['discrete_cells']['oncoprotein']

In [ ]:
plt.scatter(cx,cy,c=op)
plt.axis([-250,250,-250,250])

### Code Section 5

In [ ]:
plt.clf()
plt.scatter( cx , cy, c=op, s=30 )

plt.axis( 'image' )
plt.axis( [-250,250,-250,250] )

plt.colorbar()

plt.title( 'Cells colored by oncoprotein value' , size=20)
plt.xlabel( 'x' , size=15 ) 
plt.ylabel( 'y', size=15 )

### Code Section 6

In [ ]:
mcds = pyMCDS('output00000006.xml', 'output')
t=mcds.get_time()
cx = mcds.data['discrete_cells']['position_x']
cy = mcds.data['discrete_cells']['position_y']
op = mcds.data['discrete_cells']['oncoprotein']
plt.clf()
plt.hist( op ) 
plt.title( 'oncoprotein at t=' + \
str(t/60) + ' hours' , size=20)

### Code Section 7 

In [ ]:
cycle = mcds.data['discrete_cells']['cycle_model']
cycle = cycle.astype( int ) 

live = np.argwhere( cycle < 100 ).flatten()
dead = np.argwhere( cycle >= 100 ).flatten()

print( len( live) )
print( len(dead) )

### Code Section 8

In [ ]:
# Live and dead cell counts
n_live = len( live ); print( n_live) 
n_dead = len( dead ); print( n_dead ); 

# Mean oncoprotein in live cells only 
print( np.mean( op[live] ) )


# Let's scatter plot of only live cells  
plt.clf()
plt.scatter( cx[live],cy[live],c=op[live],s=5); 
plt.colorbar()
plt.axis('image')
plt.axis([-400,400,-400,400])
plt.title( 'Live cells colored by p at t=' +str(t/60) + ' hr',size=20)

### Code Section 8a

In [ ]:
ind = np.argwhere( (cycle<100) & (op>5) ) .flatten()
plt.clf()
plt.scatter( cx[ind], cy[ind], c=op[ind], s=10 )
plt.title( 'Live cells with p>5 at t='\
+str(t/60) + 'hr', size=20)
plt.axis('image')
plt.axis([-400,400,-400,400])
plt.colorbar()

### Code Section 9 

In [ ]:
plt.clf()
print( mcds.get_substrate_names() ); 

In [ ]:
o2 = mcds.get_concentrations( 'oxygen' );
X,Y = mcds.get_2D_mesh(); 

plt.clf()
plt.contourf(X,Y,o2[:,:,0]);
plt.colorbar()
plt.axis('image')

### Code Section 10 

In [ ]:
circle_size = 5

plt.clf()
mcds.get_substrate_names(); 

o2 = mcds.get_concentrations( 'oxygen' );
X,Y = mcds.get_2D_mesh(); 
plt.contourf(X,Y,o2[:,:,0],cmap='spring');
plt.colorbar()

plt.scatter( cx[live],cy[live],c=op[live],s=circle_size); 
plt.axis('image')
plt.axis([-400,400,-400,400])
plt.title( 'Live cells colored by p at t=' +str(t/60) + ' hr', size=20)

# let's plot dead cells as white and transparent
plt.scatter( cx[dead],cy[dead],c='w',s=circle_size, alpha=0.5 ); 

### Code Section 11

In [ ]:
# Let's get live and dead cell counts, mean p (in live cells). We need to loop overall simulation times 

last_index = 6;
live_count = np.zeros( last_index+1 ); 
dead_count = np.zeros( last_index+1 );
mean_p = np.zeros( last_index+1 );
std_p = np.zeros( last_index+1 );
times = np.zeros( last_index+1 ); 
for n in range( 0,last_index+1 ):
	filename='output'+"%08i"%n+'.xml'
	mcds=pyMCDS(filename,'output')
	times[n]= mcds.get_time()
	cycle=mcds.data['discrete_cells']['cycle_model']
	p = mcds.data['discrete_cells']['oncoprotein']
	live = np.argwhere(cycle<100).flatten()
	dead = np.argwhere(cycle>=100).flatten()
	live_count[n] = len(live)
	dead_count[n] = len(dead)
	mean_p[n] = np.mean( p[live] )
	std_p[n] = np.std( p[live] )


### Code Section 12

In [ ]:
plt.clf()
plt.plot( times, live_count , 'r-o' )
plt.plot( times, dead_count , 'b-o' ); 
plt.legend( {'live', 'dead' } ) 

In [ ]:
poly=np.polyfit( times,np.log(live_count),1) 
print( poly[0] )
# growth rate is 0th element
# in units of 1/min 
# 0.0005928392815655603

In [ ]:
plt.clf()
plt.plot(times,mean_p);
# mean increases rapidly 
# due to selection processes


### Code Section 13

In [ ]:
n = 3
filename='output'+"%08i"%n+'.xml'
mcds=pyMCDS(filename,'output')
t = mcds.get_time()
cell_type=mcds.data['discrete_cells']['cell_type']
cell_type=cell_type.astype(int)

ind1 = np.argwhere(cell_type==1).flatten(); # director 
ind2 = np.argwhere(cell_type==2).flatten(); # cargo 
ind3 = np.argwhere(cell_type==3).flatten(); # worker 

cx = mcds.data['discrete_cells']['position_x']
cy = mcds.data['discrete_cells']['position_y']

### Code Section 14

In [ ]:
circle_size=20

plt.clf()
plt.figure(figsize=(15,15))
plt.scatter(cx[ind1],cy[ind1],c='g',s=circle_size) 
plt.scatter(cx[ind2],cy[ind2],c='b',s=circle_size) 
plt.scatter(cx[ind3],cy[ind3],c='r',s=circle_size) 
plt.axis('square');

### Code Section 15

In [ ]:
mcds.get_substrate_names(); 

cs = mcds.get_concentrations( 'cargo signal' );
X,Y = mcds.get_2D_mesh(); 

plt.clf()
plt.figure(figsize=(15,15))
plt.contourf(X,Y,cs[:,:,0],cmap='gray');

plt.scatter(cx[ind1],cy[ind1],c='g',s=circle_size) 
plt.scatter(cx[ind2],cy[ind2],c='b',s=circle_size) 
plt.scatter(cx[ind3],cy[ind3],c='r',s=circle_size) 
plt.axis('square');